# Grupo 1 - Smog predicition
## Decision Tree

### Análisis y limpieza de datos

En primer lugar, realizaremos la importación de las librerías necesarias, así como del dataframe, y procederemos a su tratatmiento y limpieza.

In [ ]:
#Imports generales
import numpy as np
import pandas as pd
from pandas import Series
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

#Imports específicos
from sklearn import tree
from sklearn.metrics import recall_score, precision_score, make_scorer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from scipy.stats import sem
from sklearn.model_selection import GridSearchCV

#Visualización
import seaborn as sns
sns.set(color_codes=True)

%matplotlib inline

df = pd.read_csv('data/train.csv')

df['Gears'] = df['Transmission'].str.extract('(\d+)')
df['Gears'] = pd.to_numeric(df['Gears'], errors='coerce')
df['Transmission'] = df['Transmission'].str.extract('(\D+)')

In [ ]:
#Fuel Type
df.loc[df["Fuel Type"] == "X", "Fuel Type"] = 0
df.loc[df["Fuel Type"] == "Z", "Fuel Type"] = 1
df.loc[df["Fuel Type"] == "D", "Fuel Type"] = 2
df.loc[df["Fuel Type"] == "E", "Fuel Type"] = 3
df.loc[df["Fuel Type"] == "N", "Fuel Type"] = 4

#Transmission
df.loc[df["Transmission"] == "A", "Transmission"] = 0
df.loc[df["Transmission"] == "AM", "Transmission"] = 1
df.loc[df["Transmission"] == "AS", "Transmission"] = 2
df.loc[df["Transmission"] == "AV", "Transmission"] = 3
df.loc[df["Transmission"] == "M", "Transmission"] = 4


#Vehicle Class
df.loc[df["Vehicle Class"] == "Compact", "Vehicle Class"] = 0
df.loc[df["Vehicle Class"] == "Full-size", "Vehicle Class"] = 1
df.loc[df["Vehicle Class"] == "Mid-size", "Vehicle Class"] = 2
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 3
df.loc[df["Vehicle Class"] == "Minivan", "Vehicle Class"] = 4
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 5
df.loc[df["Vehicle Class"] == "Pickup truck: Small", "Vehicle Class"] = 6
df.loc[df["Vehicle Class"] == "Pickup truck: Standard", "Vehicle Class"] = 7
df.loc[df["Vehicle Class"] == "SUV: Small", "Vehicle Class"] = 8
df.loc[df["Vehicle Class"] == "SUV: Standard", "Vehicle Class"] = 9
df.loc[df["Vehicle Class"] == "Special purpose vehicle", "Vehicle Class"] = 10
df.loc[df["Vehicle Class"] == "Station wagon: Mid-size", "Vehicle Class"] = 11
df.loc[df["Vehicle Class"] == "Station wagon: Small", "Vehicle Class"] = 12
df.loc[df["Vehicle Class"] == "Subcompact", "Vehicle Class"] = 13
df.loc[df["Vehicle Class"] == "Two-seater", "Vehicle Class"] = 14

#Gears
df['Gears'] = df['Gears'].fillna(df['Gears'].mean())

In [ ]:
df.drop("Model Year", axis=1, inplace=True)
df.drop("Make", axis=1, inplace=True)
df.drop("Model", axis=1, inplace=True)
df.drop("Comb (mpg)", axis=1, inplace=True)
df.drop("Fuel Consumption City (L/100 km)", axis=1, inplace=True)
df.drop("Hwy (L/100 km)", axis=1, inplace=True)

df.head()

Con esto, hemos terminado el análisis y la limpieza de los datos del dataframe de entrenamiento.

### Definición y entrenamiento del modelo

En este *notebook* realizaremos el entrenamiento de un modelo de árbol de decisión, el cual entrenaremos empleando los campos 'Vehicle Class', 'Engine Size (L)', 'Cylinders', 'Transmission', 'Fuel Type', 'CO2 Emissions (g/km)' y 'Gears'.

En primer lugar separaremos el conjunto de datos en subconjuntos de entrenamiento y prueba para realizar una primera aproximación.

In [ ]:
features = ['Cylinders', 'Transmission', 'Fuel Type', 'CO2 Emissions (g/km)', 'Gears']

x = df[features].values
y = df['Smog'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=33)

# Normalización
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

Y una vez seleccionados los datos, entrenamos el modelo.

In [ ]:
model = tree.DecisionTreeClassifier(max_depth=3, random_state=1)

model.fit(x_train, y_train)

Inicialmente se usan estos hiperparámetros para el entrenamiento. Posteriormente los ajustaremos para obtener una mayor eficiencia.

### Comprobación de resultados

Para la comprobación de resultados se van a calcular varias métricas, obteniendo la eficacia de nuestro modelo.

In [ ]:
# Evaluar la Exactitud en el entrenamiento
predicted = model.predict(x_test)
expected = y_test

y_train_pred = model.predict(x_train)
print("Accuracy in training", metrics.accuracy_score(y_train, y_train_pred))

# También vamos a evaluar el error en las pruebas
y_test_pred = model.predict(x_test)
print("Accuracy in testing ", metrics.accuracy_score(y_test, y_test_pred))

Con esta primera aproximación, obtenemos una exactitud del 55% para este modelo.

In [ ]:
s_y_test = Series(y_test)
s_y_test.value_counts()

s_y_test.value_counts().head(1) / len(y_test)

Al cumplir la exactitud nula podemos deducir que el modelo no se encuentra desbalanceado en nuestro conjunto de datos.

La última medida del funcionamiento de este modelo la realizaremos mediante el F1-score:

In [ ]:
print(classification_report(expected, predicted))

Obtenemos resultados superiores al SVM, pero muy debajo de lo que debería.

Para intentar mejorar nuestros resultados, emplearemos validación mediante K-Fold a la hora de dividir el conjunto de entrenamiento y así evitar posibles sesgos.

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=33)

scores = cross_val_score(model, x, y, cv=cv)
print("Scores in every iteration", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

La puntuación media se ha mantenido casi igual al caso anterior.

### Ajuste del algoritmo

A continuación, vamos a realizar un ajuste de los hiperparámetros del algoritmo mediante *grid search*.

In [ ]:
# Conjunto de hiperparámetros a probar
tuned_hyperparameters = [{'max_depth': np.arange(3, 10),
                     'criterion': ['gini', 'entropy', 'log_loss'], 
                     'splitter': ['best', 'random'],
                     'min_samples_leaf': [2, 5, 10],
                     'class_weight':['balanced', None],
                     'max_leaf_nodes': [None, 5, 10, 20]
                    }]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print()

    if score == 'precision':
        scorer = make_scorer(precision_score, average='weighted', zero_division=0)
    elif score == 'recall':
        scorer = make_scorer(recall_score, average='weighted', zero_division=0)
    
    gs = GridSearchCV(DecisionTreeClassifier(), tuned_hyperparameters, cv=10, scoring=scorer)
    gs.fit(x_train, y_train)

    print("Best hyperparameters set found on development set:")
    print()
    print(gs.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']

    for mean_score, std_score, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean_score, std_score * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, gs.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()

### Comprobación con el algoritmo ajustado

A partir de los resultados anteriores, volvemos a entrenar el modelo mediante validación con K-Fold para comprobar la nueva media de puntuación.

In [ ]:
model = Pipeline([
        ('scaler', StandardScaler()),
        ('ds', DecisionTreeClassifier(**gs.best_params_))
])

model.fit(x_train, y_train) 

cv = KFold(10, shuffle=True, random_state=33)

scores = cross_val_score(model, x, y, cv=cv)
def mean_score(scores):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))

Con el algoritmo ajustado, obtenemos una media de puntuación del 67%, lo cual mejora un poco la primera obtenida, pero sigue siendo bastante mejorable.